In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset


In [2]:
df_text = pd.read_csv("/kaggle/input/twibot-sc-text/Text5_balanced.csv")

In [3]:
df_users = pd.read_csv("/kaggle/input/twibot-sc-text/Users_balanced.csv")

In [4]:
df_users

,description_len,protected,followers_count,following_count,tweet_count,listed_count,verified,label,id
0,26,0,45541,1206,9194,605,0,1,u2465283662
1,44,0,26,121,4883,0,0,1,u864305332075364352
2,107,0,22642,2490,12436,190,0,0,u196278935
3,149,0,2532,2306,11674,36,0,0,u731478516181565440
4,159,0,1341,1979,1881,53,0,1,u35708687
...,...,...,...,...,...,...,...,...,...
93271,98,0,1354,1355,239,72,0,0,u18134480
93272,121,0,76,20,107,2,0,0,u706703037973147652
93273,174,0,1319,2691,35369,185,0,1,u22586384
93274,55,0,2426,430,264381,27,0,1,u747814153


In [5]:
df_text

,author_id,text,label
0,u1000023384531636224,@karentatarian_ @Sorbonne_Univ_ @ISIR_labo @AN...,bot
1,u1000054188183904257,@charlieINTEL Just remove the MVP thing... [1]...,bot
2,u1000060938463170560,Dungeon #2: The Cave of Ao Shun\n\n#IndieDev #...,human
3,u1000083602456154112,"Tour-de-force by @rafacze, @ZinselmeyerB, Emma...",human
4,u1000116915082231809,Check out my NFT listing on OpenSea! https://t...,bot
...,...,...,...
93271,u999934898470342656,Chrissie highlights the 5 key characteristics ...,bot
93272,u99998808,Let's indulge in some escapism in these compli...,human
93273,u999991144384737280,First Somali Crowdfunding Platform#Support the...,bot
93274,u999993859605172224,Unilode's non-obstructive mounting solution fo...,bot


In [6]:
df_full = df_users.merge(df_text, how="left", left_on="id", right_on="author_id")

In [7]:
df_full.columns

Index(['description_len', 'protected', 'followers_count', 'following_count',
       'tweet_count', 'listed_count', 'verified', 'label_x', 'id', 'author_id',
       'text', 'label_y'],
      dtype='object')

In [8]:
df_full = df_full[['description_len', 'protected', 'followers_count', 'following_count', 'tweet_count', 'listed_count', 'verified', 'label_x', 'text', 'label_y', 'id']]

In [9]:
from transformers import RobertaTokenizer, RobertaModel
import torch

2025-04-15 19:46:12.092811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744746372.315247      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744746372.379289      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
df_0 = df_full[df_full['label_x'] == 0].sample(n=5000, random_state=42)
df_1 = df_full[df_full['label_x'] == 1].sample(n=5000, random_state=42)

df_full = pd.concat([df_0, df_1]).reset_index(drop=True)

In [11]:
# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)

texts = df_full['text'].to_list()

# Tokenize and convert to input tensors


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def get_embeds(texts, batch_size=32, device='cuda' if torch.cuda.is_available() else 'cpu'):
    # Move model to appropriate device
    model.to(device)
    model.eval()

    # Create batches of text
    embeddings = []
    
    # Tokenize in batches
    for i in range(0, len(texts), batch_size):
        # Get the current batch of texts
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize and move to the appropriate device
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Get embeddings with no gradients
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract the last hidden states
        last_hidden_state = outputs.last_hidden_state
        
        # Mean pooling to get sentence-level embeddings
        batch_embeddings = last_hidden_state.mean(dim=1)  # Shape: [batch_size, hidden_size]
        embeddings.append(batch_embeddings)

    # Stack all batches into one tensor
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings


In [13]:
embs = get_embeds(texts)

In [14]:
embs

tensor([[-0.0638,  0.0458,  0.0374,  ...,  0.0177, -0.0301,  0.0139],
        [-0.0887, -0.0281,  0.0331,  ..., -0.1511,  0.0068, -0.0102],
        [-0.0899,  0.0449,  0.0456,  ..., -0.1216, -0.0198,  0.0149],
        ...,
        [-0.0240, -0.0082,  0.0340,  ..., -0.0845, -0.0203,  0.0505],
        [-0.0381,  0.0189, -0.0119,  ..., -0.1873,  0.0284,  0.0096],
        [-0.0313, -0.0027,  0.0717,  ..., -0.0219,  0.0626,  0.0540]],
       device='cuda:0')

In [15]:
embeddings_df = pd.DataFrame(embs.cpu().numpy())

In [16]:
df_combined = pd.concat([embeddings_df.reset_index(drop=True), df_full.reset_index(drop=True)], axis=1)

In [17]:
df_combined.to_csv("robert_embs.csv", index=False)

In [18]:
df_combined

,0,1,2,3,4,5,6,7,8,9,...,protected,followers_count,following_count,tweet_count,listed_count,verified,label_x,text,label_y,id
0,-0.063839,0.045821,0.037418,-0.005265,-0.023820,-0.147985,0.041368,0.268481,0.093845,-0.051491,...,0,456,1387,1962,19,0,0,Thanks @COVIDVxProject #COVIDVxAnswers https:/...,human,u556941520
1,-0.088703,-0.028094,0.033069,0.000010,0.057710,-0.186109,-0.033170,0.113711,0.075059,-0.039108,...,0,4704,3576,7153,38,0,0,Wordle 258 3/6\n\n🟩⬛⬛⬛🟩\n🟩⬛🟨⬛🟩\n🟩🟩🟩🟩🟩 [1] G’da...,human,u201482514
2,-0.089906,0.044870,0.045571,0.020343,0.002174,-0.019310,-0.025727,0.166047,0.141430,-0.014809,...,0,24628,539,4987,239,1,0,Equestrian events are coming up in #ChesterCou...,human,u76733325
3,-0.026522,0.056313,0.005001,-0.035982,0.021892,-0.073461,0.030592,0.234562,0.041704,-0.105489,...,0,326,182,168,2,0,0,RT @ClareRayner6: Collaborative learning durin...,human,u1093362632269062145
4,-0.036116,-0.003888,0.075635,-0.069689,0.139981,-0.103635,0.013286,0.283860,0.048024,-0.130018,...,0,437,318,3356,3,0,0,@richardfbetzel Some other folks not presentin...,human,u151213852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.035099,-0.074625,0.044380,-0.082415,-0.101543,-0.162850,0.018927,0.126560,0.147278,0.028809,...,0,403,740,15576,12,0,1,RT @Louis_Tomlinson: Go on Jamie lad !! Great ...,bot,u940287056
9996,-0.065824,0.008357,0.016951,-0.005660,-0.021456,-0.150749,-0.018298,0.154561,0.045559,-0.057149,...,0,654,528,626,13,0,1,Demis Roussos - Midnight Is The Time I Need Yo...,bot,u14657335
9997,-0.024026,-0.008164,0.034027,-0.054016,-0.032265,-0.158608,0.015137,0.248405,0.035772,-0.038946,...,0,18,200,665,1,0,1,@marcorubio Then why are you sharing unconfirm...,bot,u3409139452
9998,-0.038052,0.018908,-0.011875,0.096292,-0.036807,-0.115779,0.010271,0.094737,-0.010851,-0.086748,...,0,378,672,2276,21,0,1,"@aleumbrico is going to present ""ROXANNE – A T...",bot,u632749640
